In [1]:
DATA_NAME = 'syntheticMRI2D-coronal' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_coronal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,170311.51,7.87,-0.13,-2.08,0.23,-0.21,0.01
4,7.87,269350.80,6.28,1.25,2.33,-0.61,-0.04
5,-0.13,6.28,89767.41,0.49,-0.55,0.45,0.02
6,-2.08,1.25,0.49,27726.66,-0.30,-0.10,0.09
7,0.23,2.33,-0.55,-0.30,5938.64,0.04,0.00
8,-0.21,-0.61,0.45,-0.10,0.04,762.63,-0.00
9,0.01,-0.04,0.02,0.09,0.00,-0.00,72.96


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00004,-0.00000,-0.00003,0.00001,-0.00002,0.00000
4,0.00004,1.00000,0.00004,0.00001,0.00006,-0.00004,-0.00001
5,-0.00000,0.00004,1.00000,0.00001,-0.00002,0.00005,0.00001
6,-0.00003,0.00001,0.00001,1.00000,-0.00002,-0.00002,0.00007
7,0.00001,0.00006,-0.00002,-0.00002,1.00000,0.00002,0.00000
8,-0.00002,-0.00004,0.00005,-0.00002,0.00002,1.00000,-0.00000
9,0.00000,-0.00001,0.00001,0.00007,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

15.107897995891774

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.05797093e+10 4.03312765e+09 1.06122103e+09 9.57847020e+07
 3.91205033e+06 4.87023639e+04 1.29693773e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.163769,0.984923,-0.053686,-0.014657,-0.003096,-0.000401,-0.000038
1,0.975657,0.153373,-0.153166,-0.032606,-0.006765,-0.000870,-0.000083
2,0.139286,0.075319,0.982961,-0.091690,-0.017370,-0.002213,-0.000212
3,0.041618,0.025984,0.083304,0.992330,-0.076477,-0.009230,-0.000878
4,0.011609,0.007327,0.022080,0.073514,0.994761,-0.065815,-0.006142
5,0.002214,0.001401,0.004194,0.013603,0.063988,0.993232,-0.095847
6,0.000426,0.000270,0.000808,0.002616,0.012228,0.095226,0.995377


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.1637693094891914,
 0.8466269966383664,
 0.01703882972655224,
 0.007670095491907447,
 0.005238556690262075,
 0.00676820757157226,
 0.004623303514714849]